In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams

In [4]:
df = pd.read_csv('data/tokenized_clean.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.replace(to_replace=r"\[|\]|\' ", value='', inplace=True, regex=True)
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this may allah fo...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,people receive evacuation orders in california,1
4,7,NaN,NaN,just got sent this photo from ruby as smoke f...,1


In [5]:
freqs = {}
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 4):
        for j in ngrams(tokens, n):
            j = str(j)
            freqs.setdefault(j, 0)
            freqs[j]+=1
freqs

{"('our',)": 99,
 "('deeds',)": 2,
 "('are',)": 401,
 "('the',)": 3261,
 "('reason',)": 20,
 "('of',)": 1823,
 "('this',)": 477,
 "('may',)": 88,
 "('allah',)": 4,
 "('forgive',)": 2,
 "('us',)": 163,
 "('all',)": 257,
 "('our', 'deeds')": 1,
 "('deeds', 'are')": 1,
 "('are', 'the')": 15,
 "('the', 'reason')": 2,
 "('reason', 'of')": 2,
 "('of', 'this')": 9,
 "('this', 'may')": 1,
 "('may', 'allah')": 3,
 "('allah', 'forgive')": 1,
 "('forgive', 'us')": 1,
 "('us', 'all')": 5,
 "('our', 'deeds', 'are')": 1,
 "('deeds', 'are', 'the')": 1,
 "('are', 'the', 'reason')": 1,
 "('the', 'reason', 'of')": 1,
 "('reason', 'of', 'this')": 1,
 "('of', 'this', 'may')": 1,
 "('this', 'may', 'allah')": 1,
 "('may', 'allah', 'forgive')": 1,
 "('allah', 'forgive', 'us')": 1,
 "('forgive', 'us', 'all')": 1,
 "('forest',)": 63,
 "('fire',)": 247,
 "('near',)": 54,
 "('la',)": 25,
 "('ronge',)": 1,
 "('sask',)": 1,
 "('canada',)": 9,
 "('forest', 'fire')": 8,
 "('fire', 'near')": 3,
 "('near', 'la')": 1,


In [22]:
indexer = {}
index = 0
for i in df['text']:
    tokens = nltk.tokenize.word_tokenize(i, language="english")
    for n in range(1, 3):
        for j in ngrams(tokens, n):
            j = str(j)
            if j not in indexer.keys() and freqs[j] >= 2:
                indexer[j] = index
                index+=1
indexer

{"('our',)": 0,
 "('deeds',)": 1,
 "('are',)": 2,
 "('the',)": 3,
 "('reason',)": 4,
 "('of',)": 5,
 "('this',)": 6,
 "('may',)": 7,
 "('allah',)": 8,
 "('forgive',)": 9,
 "('us',)": 10,
 "('all',)": 11,
 "('are', 'the')": 12,
 "('the', 'reason')": 13,
 "('reason', 'of')": 14,
 "('of', 'this')": 15,
 "('may', 'allah')": 16,
 "('us', 'all')": 17,
 "('forest',)": 18,
 "('fire',)": 19,
 "('near',)": 20,
 "('la',)": 21,
 "('canada',)": 22,
 "('forest', 'fire')": 23,
 "('fire', 'near')": 24,
 "('residents',)": 25,
 "('asked',)": 26,
 "('to',)": 27,
 "('shelter',)": 28,
 "('in',)": 29,
 "('place',)": 30,
 "('being',)": 31,
 "('by',)": 32,
 "('officers',)": 33,
 "('no',)": 34,
 "('other',)": 35,
 "('evacuation',)": 36,
 "('or',)": 37,
 "('orders',)": 38,
 "('expected',)": 39,
 "('shelter', 'in')": 40,
 "('in', 'place')": 41,
 "('are', 'being')": 42,
 "('no', 'other')": 43,
 "('people',)": 44,
 "('receive',)": 45,
 "('california',)": 46,
 "('evacuation', 'orders')": 47,
 "('in', 'california')"

In [20]:
len(indexer)*len(df)*4/10e9

0.2094671272

In [23]:
n_gram_onehot = np.zeros(shape=(len(df), len(indexer)), dtype=np.float32)
n_gram_onehot.shape

(7613, 16672)

In [8]:
import json
with open('indexes.sav', 'w') as f:
    f.write(json.dumps(indexer))

In [9]:
missed = 0
for i, e in enumerate(df['text']):
    tokens = nltk.tokenize.word_tokenize(e, language="english")
    matched = False
    for j in ngrams(tokens, n):
        j = str(j)
        if j in indexer.keys():
            n_gram_onehot[i][indexer[j]] = 1.0
            matched = True
    if not matched:
        missed +=1
print(missed)

494


In [10]:
labelsOneHot = np.zeros((n_gram_onehot.shape[0], 2))
for i, e in enumerate(df['target']):
    labelsOneHot[i][e]=1
labelsOneHot.shape

(7613, 2)

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [18]:
keras.utils.set_random_seed(1234)
ngram_NN = keras.Sequential()
ngram_NN.bias = 0.2
ngram_NN.add(keras.layers.Input(16672))

ngram_NN.add(keras.layers.Reshape((521, 32)))

ngram_NN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))#, padding="causal"))
ngram_NN.add(keras.layers.MaxPool1D(2))

ngram_NN.add(keras.layers.Flatten())

ngram_NN.add(keras.layers.Dense(512, activation='relu'))
ngram_NN.add(keras.layers.Dense(128,  activation='relu'))
ngram_NN.add(keras.layers.Dense(32,  activation='relu'))
ngram_NN.add(keras.layers.Dense(8,  activation='relu'))



ngram_NN.add(keras.layers.Dropout(.5))


ngram_NN.add(keras.layers.Dense(2, activation='softmax'))
ngram_NN.output_shape

ValueError: Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 521, 32)

In [13]:
callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]

In [14]:
ngram_NN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [17]:
ngram_NN.fit(n_gram_onehot, labelsOneHot, epochs=25, callbacks=callbacks)

Epoch 1/25
238/238 [==============================] - 7s 31ms/step - loss: 0.1786 - accuracy: 0.9020
Epoch 2/25
238/238 [==============================] - 7s 28ms/step - loss: 0.1816 - accuracy: 0.9016
Epoch 3/25
238/238 [==============================] - 7s 28ms/step - loss: 0.1915 - accuracy: 0.8981
Epoch 4/25
238/238 [==============================] - 7s 28ms/step - loss: 0.1923 - accuracy: 0.8957
Epoch 5/25
238/238 [==============================] - 7s 28ms/step - loss: 0.1814 - accuracy: 0.8999
Epoch 6/25
238/238 [==============================] - 7s 28ms/step - loss: 0.1758 - accuracy: 0.9017
Epoch 7/25
238/238 [==============================] - 8s 35ms/step - loss: 0.1684 - accuracy: 0.9083
Epoch 8/25
238/238 [==============================] - 8s 33ms/step - loss: 0.1849 - accuracy: 0.8957
Epoch 9/25
238/238 [==============================] - 8s 32ms/step - loss: 0.1814 - accuracy: 0.9016
Epoch 10/25
238/238 [==============================] - 8s 33ms/step - loss: 0.1892 - accura